In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import folium
from sklearn.preprocessing import LabelEncoder
from geopy.geocoders import Yandex
from geopy.distance import vincenty
from time import time
from tqdm import tqdm

In [121]:
train = pd.read_csv('data/train_set.csv')
test = pd.read_csv('data/test_set.csv')

train.rename(columns={ 'pos_adress_lat' : 'pos_address_lat', 
                       'pos_adress_lon' : 'pos_address_lon'}, inplace=True)

D:\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (1,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
D:\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (1,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [123]:
# Заполнение пропуска координат дома
customer = train['customer_id'][train['home_add_lon'].isnull() == True].unique()[0]
train['home_add_lon'][train['customer_id'] == customer] = train['home_add_lon'][train['customer_id'] == customer].fillna(method='bfill').fillna(method='ffill')
train['home_add_lat'][train['customer_id'] == customer] = train['home_add_lat'][train['customer_id'] == customer].fillna(method='bfill').fillna(method='ffill')

#Удалить клиентов, у которых нет работы
train.drop(train[train['work_add_lon'] == 0].index,inplace=True)

# Удалить транзакции, где нет адреса координат atm, pos и terminal_id (в трейне это все строчки, где нет terminal_id)
train.drop(train[train['terminal_id'].isnull() == True].index, axis=0, inplace=True)

###########################################################################################
# Подсчет частот адресов
atm_address_count = train['atm_address'].value_counts()

# Удаление транзакций с atm_address = "RADNAYA\             RUS,445" (по нему координат нет)
# train.drop(train[train['atm_address'] == atm_address_count.index[45]].index, axis=0, inplace=True)

# Замена 'Подольск, ул. Полевановская, д. 9, (Заезд с ул. Орджоникидзе д. 25)' на 'Подольск, ул. Полевановская, д. 9'
train['atm_address'].replace(to_replace=atm_address_count.index[46], value='Подольск, ул. Полевановская, д. 9', inplace=True)

############################################################################################
# Адреса с '\' в названии (частоты)
backslash_atm_address_count = train['atm_address'][train['atm_address'].str.contains("\\\\") == True].value_counts()

# Замены до частоты 100 более-менее читаемых адресов
train['atm_address'].replace(to_replace=backslash_atm_address_count.index[1], value='улица Маросейка, 3/13с1, Москва', inplace=True)
train['atm_address'].replace(to_replace=backslash_atm_address_count.index[9], value='улица Савушкина, 141, Санкт-Петербург', inplace=True)
train['atm_address'].replace(to_replace=backslash_atm_address_count.index[12], value='Новомытищинский проспект, 34, Москва', inplace=True)
train['atm_address'].replace(to_replace=backslash_atm_address_count.index[14], value='проспект Большевиков, 18, Санкт-Петербург', inplace=True)
train['atm_address'].replace(to_replace=backslash_atm_address_count.index[15], value='Марксистская улица, 1, Москва', inplace=True)
train['atm_address'].replace(to_replace=backslash_atm_address_count.index[16], value='Гражданский проспект, 41к2, Санкт-Петербург', inplace=True)
train['atm_address'].replace(to_replace=backslash_atm_address_count.index[18], value='улица Грекова, 8, Москва', inplace=True)
train['atm_address'].replace(to_replace=backslash_atm_address_count.index[23], value='Ракетный бульвар, 16, Москва', inplace=True)
train['atm_address'].replace(to_replace=backslash_atm_address_count.index[27], value='Новочеркасский проспект, 43/17, Санкт-Петербург', inplace=True)

# Остальные удалить
train.drop(train[train['atm_address'].str.contains("\\\\") == True].index, inplace=True)

############################################################################################
# Для удобства анализа сделаю labelencoding на customer_id и terminal_id
label_encoder = LabelEncoder()
train['customer_id'] = label_encoder.fit_transform(train['customer_id'])
train['terminal_id'] = label_encoder.fit_transform(train['terminal_id'])

#############################################################################################
# Обнуление NaN адресов atm и pos
train[['atm_address', 'atm_address_lat', 'atm_address_lon',
       'pos_address', 'pos_address_lat', 'pos_address_lon']] = train[['atm_address', 'atm_address_lat', 'atm_address_lon', 
                                                                      'pos_address', 'pos_address_lat', 'pos_address_lon']].fillna(0)
#######################################################################################################

# у 13 pos адресов координаты разные - исправляем
wrong_pos_address = []
with open('pos_address.txt', 'r') as file:
    for line in file:
        wrong_pos_address.append(line.strip())
        
geolocator = Yandex()

#0
location = geolocator.geocode('B. KAMENSCHIKI, 4MOSCOW')
#pos = train[train['pos_address'] == wrong_pos_address[0]]
#pos = pos.groupby(['pos_address_lat', 'pos_address_lon']).size().reset_index().rename(columns={0:'count'})

train['pos_address_lat'][train['pos_address'] == wrong_pos_address[0]] = location.latitude
train['pos_address_lon'][train['pos_address'] == wrong_pos_address[0]] = location.longitude

#1
train['pos_address_lat'][train['pos_address'] == wrong_pos_address[1]] = 59.2243102
train['pos_address_lon'][train['pos_address'] == wrong_pos_address[1]] = 39.8657153

#2 под вопросом, потому что обе точки имеют место быть
#train['pos_address_lat'][train['pos_address'] == wrong_pos_address[2]] = 55.7955715
#train['pos_address_lon'][train['pos_address'] == wrong_pos_address[2]] = 37.5937033

#3 
# тоже обе точки могут быть

#4
location = geolocator.geocode("5-1A, SUSCHEVSKIY VALMOSKVA")
train['pos_address_lat'][train['pos_address'] == wrong_pos_address[4]] = location.latitude
train['pos_address_lon'][train['pos_address'] == wrong_pos_address[4]] = location.longitude

#5
location = geolocator.geocode("'KOMMUNISTICHESKAYA STR 1MYTISCHY")
train['pos_address_lat'][train['pos_address'] == wrong_pos_address[5]] = location.latitude
train['pos_address_lon'][train['pos_address'] == wrong_pos_address[5]] = location.longitude

#6
location = geolocator.geocode("MIRA STR 51MYTISCHY")
train['pos_address_lat'][train['pos_address'] == wrong_pos_address[6]] = location.latitude
train['pos_address_lon'][train['pos_address'] == wrong_pos_address[6]] = location.longitude

#7
location = geolocator.geocode("42, SCHUKINSKAYA MOSKVA")
train['pos_address_lat'][train['pos_address'] == wrong_pos_address[7]] = location.latitude
train['pos_address_lon'][train['pos_address'] == wrong_pos_address[7]] = location.longitude

#8
#нормальная, отличие в 8 незначящем знаке

#9
location = geolocator.geocode("2, VETERANOV MYTISCHI")
train['pos_address_lat'][train['pos_address'] == wrong_pos_address[9]] = location.latitude
train['pos_address_lon'][train['pos_address'] == wrong_pos_address[9]] = location.longitude

#10
location = geolocator.geocode("55 BOLSHOY SAMPSONIEVSKIY PR SANKT-PETERBURG")
train['pos_address_lat'][train['pos_address'] == wrong_pos_address[10]] = location.latitude
train['pos_address_lon'][train['pos_address'] == wrong_pos_address[10]] = location.longitude

#11
location = geolocator.geocode("20 BOLSHOY SAMPSONIEVSKIY PRSANKT-PETERBURG")
train['pos_address_lat'][train['pos_address'] == wrong_pos_address[11]] = location.latitude
train['pos_address_lon'][train['pos_address'] == wrong_pos_address[11]] = location.longitude

#12
location = geolocator.geocode("Счастливая улица, 11, Усады")
train['pos_address_lat'][train['pos_address'] == wrong_pos_address[12]] = location.latitude
train['pos_address_lon'][train['pos_address'] == wrong_pos_address[12]] = location.longitude

D:\Miniconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Miniconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
D:\Miniconda3\lib\site-packages\ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Miniconda3\lib\site-packages\ipykernel_launcher.py:66: Settin

#### Сначала проверим home/work координаты

In [17]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1144721 entries, 0 to 1224733
Data columns (total 18 columns):
amount              1144721 non-null float64
atm_address         1144721 non-null object
atm_address_lat     1144721 non-null float64
atm_address_lon     1144721 non-null float64
city                1144712 non-null object
country             1144721 non-null object
currency            1144677 non-null float64
customer_id         1144721 non-null int64
home_add_lat        1144705 non-null float64
home_add_lon        1144705 non-null float64
mcc                 1144721 non-null int64
pos_address         1144721 non-null object
pos_address_lat     1144721 non-null float64
pos_address_lon     1144721 non-null float64
terminal_id         1144721 non-null int64
transaction_date    1144677 non-null object
work_add_lat        625352 non-null float64
work_add_lon        625352 non-null float64
dtypes: float64(10), int64(3), object(5)
memory usage: 165.9+ MB


In [8]:
train[(train['home_add_lat']==0) | (train['home_add_lon']==0)]

,amount,atm_address,atm_address_lat,atm_address_lon,city,country,currency,customer_id,home_add_lat,home_add_lon,mcc,pos_address,pos_address_lat,pos_address_lon,terminal_id,transaction_date,work_add_lat,work_add_lon


In [21]:
train[train['home_add_lon'] != 0].shape

(1144721, 18)

In [23]:
train['home_add_lon'].isnull().value_counts()

False    1144705
True          16
Name: home_add_lon, dtype: int64

In [28]:
train[train['home_add_lon'].isnull() == True]

,amount,atm_address,atm_address_lat,atm_address_lon,city,country,currency,customer_id,home_add_lat,home_add_lon,mcc,pos_address,pos_address_lat,pos_address_lon,terminal_id,transaction_date,work_add_lat,work_add_lon
766934,2.633742,0,0.0,0.0,NOVOROSSIYSK,RUS,643.0,1167,NaN,NaN,5912,224 DZERZHINSKOGO STRNOVOROSSIYSK353900 RUSRUS,44.675740,37.780223,140887,2017-10-15,44.734,37.799
766936,3.255411,0,0.0,0.0,NOVOROSSIYSK,RUS,643.0,1167,NaN,NaN,5411,1 GRIGOREVA STRNOVOROSSIYSK353900 RUSRUS,44.669672,37.782206,159570,2017-08-03,44.734,37.799
766938,3.128106,0,0.0,0.0,NOVOROSSIYSK,RUS,643.0,1167,NaN,NaN,5411,1 GRIGOREVA STRNOVOROSSIYSK353900 RUSRUS,44.669672,37.782206,119346,2017-04-16,44.734,37.799
766940,3.751279,0,0.0,0.0,Novorossiysk,RUS,643.0,1167,NaN,NaN,5211,"ul.Pionerskay,d.23a,k.2Novorossiysk353922 R...",54.949031,20.238433,168095,2017-04-17,44.734,37.799
766942,2.836951,0,0.0,0.0,ENEM,RUS,643.0,1167,NaN,NaN,5983,22 KRASNODAR-NOVOROSSIYSK SHENEM385130 RUSRUS,44.949810,38.927252,24994,2017-07-07,44.734,37.799
766944,3.273269,0,0.0,0.0,NOVOROSSIYSK,RUS,643.0,1167,NaN,NaN,5411,48 LENINA PR-TNOVOROSSIYSK350000 RUSRUS,44.691961,37.789117,27351,2017-05-19,44.734,37.799
766946,2.018740,0,0.0,0.0,NOVOROSSIYSK,RUS,643.0,1167,NaN,NaN,5411,204-21 DZERZHINSKOGO PR-TNOVOROSSIYSK353900 ...,44.677703,37.779727,24559,2017-08-26,44.734,37.799
766948,2.742684,0,0.0,0.0,NOVOROSSIIYSK,RUS,643.0,1167,NaN,NaN,5411,"217, DZERZHINSKOGONOVOROSSIIYSK353915 RUSRUS",44.678495,37.779635,71118,2017-04-17,44.734,37.799
766950,1.785204,0,0.0,0.0,NOVOROSSIYSK,RUS,643.0,1167,NaN,NaN,5912,210 DZERZHINSKOGO PR-TNOVOROSSIYSK353900 RU...,44.678676,37.780864,127619,2017-04-17,44.734,37.799
766952,2.878566,0,0.0,0.0,NOVOROSSIYSK,RUS,643.0,1167,NaN,NaN,5411,48 LENINA PR-TNOVOROSSIYSK350000 RUSRUS,44.691961,37.789117,154432,2017-08-17,44.734,37.799


In [29]:
train[train['customer_id'] == 1167]

,amount,atm_address,atm_address_lat,atm_address_lon,city,country,currency,customer_id,home_add_lat,home_add_lon,mcc,pos_address,pos_address_lat,pos_address_lon,terminal_id,transaction_date,work_add_lat,work_add_lon
766934,2.633742,0,0.000,0.000,NOVOROSSIYSK,RUS,643.0,1167,NaN,NaN,5912,224 DZERZHINSKOGO STRNOVOROSSIYSK353900 RUSRUS,44.675740,37.780223,140887,2017-10-15,44.734,37.799
766935,2.633742,0,0.000,0.000,NOVOROSSIYSK,RUS,643.0,1167,44.675,37.781,5912,224 DZERZHINSKOGO STRNOVOROSSIYSK353900 RUSRUS,44.675740,37.780223,140887,2017-10-15,44.734,37.799
766936,3.255411,0,0.000,0.000,NOVOROSSIYSK,RUS,643.0,1167,NaN,NaN,5411,1 GRIGOREVA STRNOVOROSSIYSK353900 RUSRUS,44.669672,37.782206,159570,2017-08-03,44.734,37.799
766937,3.255411,0,0.000,0.000,NOVOROSSIYSK,RUS,643.0,1167,44.675,37.781,5411,1 GRIGOREVA STRNOVOROSSIYSK353900 RUSRUS,44.669672,37.782206,159570,2017-08-03,44.734,37.799
766938,3.128106,0,0.000,0.000,NOVOROSSIYSK,RUS,643.0,1167,NaN,NaN,5411,1 GRIGOREVA STRNOVOROSSIYSK353900 RUSRUS,44.669672,37.782206,119346,2017-04-16,44.734,37.799
766939,3.128106,0,0.000,0.000,NOVOROSSIYSK,RUS,643.0,1167,44.675,37.781,5411,1 GRIGOREVA STRNOVOROSSIYSK353900 RUSRUS,44.669672,37.782206,119346,2017-04-16,44.734,37.799
766940,3.751279,0,0.000,0.000,Novorossiysk,RUS,643.0,1167,NaN,NaN,5211,"ul.Pionerskay,d.23a,k.2Novorossiysk353922 R...",54.949031,20.238433,168095,2017-04-17,44.734,37.799
766941,3.751279,0,0.000,0.000,Novorossiysk,RUS,643.0,1167,44.675,37.781,5211,"ul.Pionerskay,d.23a,k.2Novorossiysk353922 R...",54.949031,20.238433,168095,2017-04-17,44.734,37.799
766942,2.836951,0,0.000,0.000,ENEM,RUS,643.0,1167,NaN,NaN,5983,22 KRASNODAR-NOVOROSSIYSK SHENEM385130 RUSRUS,44.949810,38.927252,24994,2017-07-07,44.734,37.799
766943,2.836951,0,0.000,0.000,ENEM,RUS,643.0,1167,44.675,37.781,5983,22 KRASNODAR-NOVOROSSIYSK SHENEM385130 RUSRUS,44.949810,38.927252,24994,2017-07-07,44.734,37.799


In [ ]:
train['customer_id'].fillna( inplace=True)

In [51]:
a = train['home_add_lat'][train['customer_id'] == 1167][:11]
a

766934       NaN
766935    44.675
766936       NaN
766937    44.675
766938       NaN
766939    44.675
766940       NaN
766941    44.675
766942       NaN
766943    44.675
766944       NaN
Name: home_add_lat, dtype: float64

In [52]:
a.fillna(method = 'bfill').fillna(method = 'ffill')

766934    44.675
766935    44.675
766936    44.675
766937    44.675
766938    44.675
766939    44.675
766940    44.675
766941    44.675
766942    44.675
766943    44.675
766944    44.675
Name: home_add_lat, dtype: float64

In [89]:
customer = train['customer_id'][train['home_add_lon'].isnull() == True].unique()[0]
train['home_add_lon'][train['customer_id'] == customer] = train['home_add_lon'][train['customer_id'] == customer].fillna(method='bfill').fillna(method='ffill')
train['home_add_lat'][train['customer_id'] == customer] = train['home_add_lat'][train['customer_id'] == customer].fillna(method='bfill').fillna(method='ffill')

AttributeError: 'NoneType' object has no attribute 'fillna'

In [101]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1144721 entries, 0 to 1224733
Data columns (total 18 columns):
amount              1144721 non-null float64
atm_address         1144721 non-null object
atm_address_lat     1144721 non-null float64
atm_address_lon     1144721 non-null float64
city                1144712 non-null object
country             1144721 non-null object
currency            1144677 non-null float64
customer_id         1144721 non-null int64
home_add_lat        1144721 non-null float64
home_add_lon        1144721 non-null float64
mcc                 1144721 non-null int64
pos_address         1144721 non-null object
pos_address_lat     1144721 non-null float64
pos_address_lon     1144721 non-null float64
terminal_id         1144721 non-null int64
transaction_date    1144677 non-null object
work_add_lat        625352 non-null float64
work_add_lon        625352 non-null float64
dtypes: float64(10), int64(3), object(5)
memory usage: 205.9+ MB


In [124]:
train[['work_add_lat', 'work_add_lon']] = train[['work_add_lat', 'work_add_lon']].fillna(0)
train[train['work_add_lon'] == 0]

,amount,atm_address,atm_address_lat,atm_address_lon,city,country,currency,customer_id,home_add_lat,home_add_lon,mcc,pos_address,pos_address_lat,pos_address_lon,terminal_id,transaction_date,work_add_lat,work_add_lon
11,0.299632,0,0.000,0.000,NOVOSIBIRSK,RUS,643.0,642,54.993,82.846,5814,5 KARLA MARKSA SQNOVOSIBIRSK630000 RUSRUS,54.982358,82.892559,145748,2017-06-16,0.0,0.0
12,3.173338,0,0.000,0.000,NOVOSIBIRSK,RUS,643.0,642,54.993,82.846,5812,17 KRASNYY PR-TNOVOSIBIRSK630000 RUSRUS,55.026997,82.920634,21758,2017-07-16,0.0,0.0
13,2.636277,0,0.000,0.000,NOVOSIBIRSK,RUS,643.0,642,54.993,82.846,5814,11-3 VATUTINA STRNOVOSIBIRSK630000 RUSRUS,54.964984,82.928976,105533,2017-05-13,0.0,0.0
14,2.774078,0,0.000,0.000,NOVOSIBIRSK,RUS,643.0,642,54.993,82.846,5813,"UL.KAMENSKAYA,1ANOVOSIBIRSK630099 RUSRUS",55.025451,82.926714,174029,2017-09-27,0.0,0.0
15,2.764726,0,0.000,0.000,NOVOSIBIRSK,RUS,643.0,642,54.993,82.846,5499,40 KOTOVSKOGO STRNOVOSIBIRSK630108 RUSRUS,54.988900,82.862242,109405,2017-05-28,0.0,0.0
16,2.905196,0,0.000,0.000,NOVOSIBIRSK,RUS,643.0,642,54.993,82.846,5814,29-1 KRASNYY PR-T BLDNOVOSIBIRSK630099 RUSRUS,55.031843,82.919409,77082,2017-08-09,0.0,0.0
17,2.082621,0,0.000,0.000,NOVOSIBIRSK,RUS,643.0,642,54.993,82.846,5912,29 KRASNYY PR-TNOVOSIBIRSK630000 RUSRUS,55.031051,82.919158,6173,2017-05-14,0.0,0.0
18,1.289195,0,0.000,0.000,NOVOSIBIRSK,RUS,643.0,642,54.993,82.846,4111,"ST.M.PLOCSHAD LENINA, V2NOVOSIBIRSK630099 R...",53.978282,79.243318,87087,2017-08-09,0.0,0.0
19,1.318607,0,0.000,0.000,NOVOSIBIRSK,RUS,643.0,642,54.993,82.846,4111,"ST.M.PLOCSHAD LENINA, V2NOVOSIBIRSK630099 R...",53.978282,79.243318,87087,2017-07-16,0.0,0.0
20,2.849170,0,0.000,0.000,NOVOSIBIRSK,RUS,643.0,642,54.993,82.846,5411,"2,TROLLEYNAYANOVOSIBIRSK630108 RUSRUS",54.970228,82.861248,112225,2017-07-03,0.0,0.0


In [185]:
print(len(train['customer_id'][train['work_add_lon'] == 0].unique()))
work = train['customer_id'][train['work_add_lon'] == 0].unique()

4842


In [198]:
train[train['work_add_lon'] == 0]

,amount,atm_address,atm_address_lat,atm_address_lon,city,country,currency,customer_id,home_add_lat,home_add_lon,mcc,pos_address,pos_address_lat,pos_address_lon,terminal_id,transaction_date,work_add_lat,work_add_lon
11,0.299632,0,0.000,0.000,NOVOSIBIRSK,RUS,643.0,642,54.993,82.846,5814,5 KARLA MARKSA SQNOVOSIBIRSK630000 RUSRUS,54.982358,82.892559,145748,2017-06-16,0.0,0.0
12,3.173338,0,0.000,0.000,NOVOSIBIRSK,RUS,643.0,642,54.993,82.846,5812,17 KRASNYY PR-TNOVOSIBIRSK630000 RUSRUS,55.026997,82.920634,21758,2017-07-16,0.0,0.0
13,2.636277,0,0.000,0.000,NOVOSIBIRSK,RUS,643.0,642,54.993,82.846,5814,11-3 VATUTINA STRNOVOSIBIRSK630000 RUSRUS,54.964984,82.928976,105533,2017-05-13,0.0,0.0
14,2.774078,0,0.000,0.000,NOVOSIBIRSK,RUS,643.0,642,54.993,82.846,5813,"UL.KAMENSKAYA,1ANOVOSIBIRSK630099 RUSRUS",55.025451,82.926714,174029,2017-09-27,0.0,0.0
15,2.764726,0,0.000,0.000,NOVOSIBIRSK,RUS,643.0,642,54.993,82.846,5499,40 KOTOVSKOGO STRNOVOSIBIRSK630108 RUSRUS,54.988900,82.862242,109405,2017-05-28,0.0,0.0
16,2.905196,0,0.000,0.000,NOVOSIBIRSK,RUS,643.0,642,54.993,82.846,5814,29-1 KRASNYY PR-T BLDNOVOSIBIRSK630099 RUSRUS,55.031843,82.919409,77082,2017-08-09,0.0,0.0
17,2.082621,0,0.000,0.000,NOVOSIBIRSK,RUS,643.0,642,54.993,82.846,5912,29 KRASNYY PR-TNOVOSIBIRSK630000 RUSRUS,55.031051,82.919158,6173,2017-05-14,0.0,0.0
18,1.289195,0,0.000,0.000,NOVOSIBIRSK,RUS,643.0,642,54.993,82.846,4111,"ST.M.PLOCSHAD LENINA, V2NOVOSIBIRSK630099 R...",53.978282,79.243318,87087,2017-08-09,0.0,0.0
19,1.318607,0,0.000,0.000,NOVOSIBIRSK,RUS,643.0,642,54.993,82.846,4111,"ST.M.PLOCSHAD LENINA, V2NOVOSIBIRSK630099 R...",53.978282,79.243318,87087,2017-07-16,0.0,0.0
20,2.849170,0,0.000,0.000,NOVOSIBIRSK,RUS,643.0,642,54.993,82.846,5411,"2,TROLLEYNAYANOVOSIBIRSK630108 RUSRUS",54.970228,82.861248,112225,2017-07-03,0.0,0.0


In [192]:
train

,amount,atm_address,atm_address_lat,atm_address_lon,city,country,currency,customer_id,home_add_lat,home_add_lon,mcc,pos_address,pos_address_lat,pos_address_lon,terminal_id,transaction_date,work_add_lat,work_add_lon
0,2.884034,0,0.000,0.000,ST PETERSBURG,RUS,643.0,617,59.851,30.232,5261,0,59.844072,30.179153,13128,2017-07-15,59.847,30.177
1,2.775633,0,0.000,0.000,ST PETERSBURG,RUS,643.0,617,59.851,30.232,5261,0,59.844072,30.179153,174451,2017-10-27,59.847,30.177
2,3.708368,0,0.000,0.000,St Petersburg,RUS,643.0,617,59.851,30.232,5992,"PR.MARSHALA ZHUKOVA,31St Petersburg190000 7...",59.858198,30.229024,166680,2017-10-03,59.847,30.177
3,2.787498,0,0.000,0.000,ST PETERSBURG,RUS,643.0,617,59.851,30.232,5261,0,59.844072,30.179153,80932,2017-09-09,59.847,30.177
4,2.892510,0,0.000,0.000,ST PETERSBURG,RUS,643.0,617,59.851,30.232,5261,0,59.844072,30.179153,4159,2017-07-06,59.847,30.177
5,2.909018,0,0.000,0.000,ST PETERSBURG,RUS,643.0,617,59.851,30.232,5261,0,59.844072,30.179153,130724,2017-08-23,59.847,30.177
6,2.801228,0,0.000,0.000,ST PETERSBURG,RUS,643.0,617,59.851,30.232,5261,0,59.844072,30.179153,4159,2017-07-29,59.847,30.177
7,2.838200,0,0.000,0.000,ST PETERSBURG,RUS,643.0,617,59.851,30.232,5261,0,59.844072,30.179153,4159,2017-07-15,59.847,30.177
8,3.264740,0,0.000,0.000,ST PETERSBURG,RUS,643.0,617,59.851,30.232,5261,0,59.844072,30.179153,141047,2017-08-18,59.847,30.177
9,3.118792,0,0.000,0.000,ST PETERSBURG,RUS,643.0,617,59.851,30.232,5261,0,59.844072,30.179153,95505,2017-05-13,59.847,30.177


In [201]:
train.drop(train[train['work_add_lon'] == 0].index,inplace=True)

In [188]:
train[train['customer_id'] == [1,2]]

TypeError: invalid type comparison

In [136]:
train[train['customer_id'] == 3]

,amount,atm_address,atm_address_lat,atm_address_lon,city,country,currency,customer_id,home_add_lat,home_add_lon,mcc,pos_address,pos_address_lat,pos_address_lon,terminal_id,transaction_date,work_add_lat,work_add_lon
140106,3.009851,0,0.0,0.0,LESNOY GORODO,RUS,643.0,3,55.742,37.575,5411,3 ENERGETIKOV STRLESNOY GORODO143000 RUSRUS,55.637990,37.222875,80960,2017-07-30,0.0,0.0
140107,3.070647,0,0.0,0.0,MOSCOW,RUS,643.0,3,55.742,37.575,5411,"KUTUZOVSKIY PR-T, DOM 9, KORPUS 1MOSCOW121248 ...",55.689295,37.537250,30576,2017-08-04,0.0,0.0
140108,3.224442,0,0.0,0.0,LESNOY GORODO,RUS,643.0,3,55.742,37.575,5411,3 ENERGETIKOV STRLESNOY GORODO143000 RUSRUS,55.637990,37.222875,138056,2017-07-08,0.0,0.0
140109,2.102438,0,0.0,0.0,YUDINO,RUS,643.0,3,55.742,37.575,5411,"55-E,YUDINO143081 RUSRUS",59.008992,51.009950,34622,2017-08-03,0.0,0.0
140110,2.723468,0,0.0,0.0,LESNOY GORODO,RUS,643.0,3,55.742,37.575,5912,"1, ENERGETIKOVLESNOY GORODO143080 RUSRUS",55.637167,37.221389,6650,2017-08-05,0.0,0.0
140111,2.676597,0,0.0,0.0,MOSCOW,RUS,643.0,3,55.742,37.575,5411,16 BOL. DOROGOMILOVSKAYA STRMOSCOW121059 RU...,55.746720,37.557262,62377,2017-08-06,0.0,0.0
140112,2.782452,0,0.0,0.0,MOSCOW,RUS,643.0,3,55.742,37.575,5411,16 BOL. DOROGOMILOVSKAYA STRMOSCOW121059 RU...,55.746720,37.557262,3983,2017-07-10,0.0,0.0
140113,3.375377,0,0.0,0.0,MOSCOW,RUS,643.0,3,55.742,37.575,5411,16 BOL. DOROGOMILOVSKAYA STRMOSCOW121059 RU...,55.746720,37.557262,3983,2017-07-09,0.0,0.0
140114,3.359777,0,0.0,0.0,MOSCOW,RUS,643.0,3,55.742,37.575,5411,"KUTUZOVSKIY PR-T, DOM 9, KORPUS 1MOSCOW121248 ...",55.689295,37.537250,30576,2017-07-28,0.0,0.0
140115,1.928986,0,0.0,0.0,LESNOJ GORODO,RUS,643.0,3,55.742,37.575,4111,"VOKZALNAYA, 2LESNOJ GORODO143080 RUSRUS",55.625555,37.215264,74406,2017-08-01,0.0,0.0


In [178]:
work2 = train.groupby('customer_id')['work_add_lat'].sum()
work2[:5]

customer_id
0    4428.765
1       0.000
2    2089.354
3       0.000
4       0.000
Name: work_add_lat, dtype: float64

In [182]:
print(len(work2[work2==0]))
work2[work2==0].index

4842


Int64Index([   1,    3,    4,    6,    7,    9,   11,   13,   17,   18,
            ...
            9983, 9984, 9986, 9987, 9988, 9991, 9992, 9994, 9996, 9997],
           dtype='int64', name='customer_id', length=4842)

In [153]:
a = np.array(work[work==0].index)

In [155]:
a[0]

1

In [166]:
np.where(train['customer_id'] == work[work==0].index)

ValueError: Lengths must match to compare

In [165]:
train[train['customer_id'] == ]

TypeError: invalid type comparison

In [142]:
work0 = train.groupby('customer_id')['work_add_lat'].sum()
train.drop([work0[work0==0].index],inplace=True)

TypeError: unhashable type: 'Int64Index'

In [115]:
train[train['customer_id'] == 1]

,amount,atm_address,atm_address_lat,atm_address_lon,city,country,currency,customer_id,home_add_lat,home_add_lon,mcc,pos_address,pos_address_lat,pos_address_lon,terminal_id,transaction_date,work_add_lat,work_add_lon
718977,2.785329,0,0.000,0.000,MOSCOW,RUS,643.0,1,55.799,37.388,5814,42 SHCHUKINSKAYA STRMOSCOW123007 RUSRUS,55.809300,37.465638,35877,2017-05-20,NaN,NaN
718978,3.105841,0,0.000,0.000,MOSCOW,RUS,643.0,1,55.799,37.388,5411,7 TALLINSKAYA STRMOSCOW141000 RUSRUS,55.799838,37.396825,32813,2017-04-09,NaN,NaN
718979,2.359619,0,0.000,0.000,MOSCOW,RUS,643.0,1,55.799,37.388,5411,3 2-Y NOVOPODMOSKOVNYY BYSTRMOSCOW141000 RU...,55.822604,37.524894,107638,2017-07-17,NaN,NaN
718980,2.712781,0,0.000,0.000,MOSCOW,RUS,643.0,1,55.799,37.388,5541,"AKADEMIKA VARGI, VL.13, STR.1MOSCOW117133 R...",55.631563,37.479914,159844,2017-08-11,NaN,NaN
718981,1.483227,0,0.000,0.000,MOSCOW,RUS,643.0,1,55.799,37.388,5411,2 AKADEMIKA ANOKHINA STR BLDMOSCOW141000 RU...,55.670364,37.477274,78983,2017-08-16,NaN,NaN
718982,2.770085,0,0.000,0.000,MOSCOW,RUS,643.0,1,55.799,37.388,5992,PRESNENSKAYA EMB.MOSCOW123317 RUSRUS,55.746734,37.540920,38832,2017-09-29,NaN,NaN
718983,1.979379,0,0.000,0.000,MOSKVA,RUS,643.0,1,55.799,37.388,5411,"10-S, PRESNENSKAYAMOSKVA123317 RUSRUS",55.746580,37.536301,136294,2017-04-06,NaN,NaN
718984,2.994524,0,0.000,0.000,MOSCOW,RUS,643.0,1,55.799,37.388,4111,"PRESNENSKAYA NAB., 2MOSCOW123456 RUSRUS",55.749044,37.539355,107125,2017-09-11,NaN,NaN
718985,2.760673,0,0.000,0.000,MOSCOW,RUS,643.0,1,55.799,37.388,5912,10 TALLINSKAYA STRMOSCOW123458 RUSRUS,55.798714,37.396852,41421,2017-09-20,NaN,NaN
718986,2.619281,0,0.000,0.000,MOSKVA,RUS,643.0,1,55.799,37.388,5411,"10-S, PRESNENSKAYAMOSKVA123317 RUSRUS",55.746580,37.536301,2869,2017-10-18,NaN,NaN


In [174]:
len(b)

4842

In [176]:
len(work[work==0])

9953

In [204]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 625352 entries, 0 to 1224733
Data columns (total 18 columns):
amount              625352 non-null float64
atm_address         625352 non-null object
atm_address_lat     625352 non-null float64
atm_address_lon     625352 non-null float64
city                625349 non-null object
country             625352 non-null object
currency            625351 non-null float64
customer_id         625352 non-null int64
home_add_lat        625352 non-null float64
home_add_lon        625352 non-null float64
mcc                 625352 non-null int64
pos_address         625352 non-null object
pos_address_lat     625352 non-null float64
pos_address_lon     625352 non-null float64
terminal_id         625352 non-null int64
transaction_date    625351 non-null object
work_add_lat        625352 non-null float64
work_add_lon        625352 non-null float64
dtypes: float64(10), int64(3), object(5)
memory usage: 90.7+ MB
